# Upload Specification file in each subject folder:
### From: Frink /HCP_Data/HCP_BedpostData/
### To: Candygram 
http://candygram.neurology.emory.edu:8080/#collection/5c48db6de62914004de6350f/folder/5c743889e62914004d6f974c

In [ ]:
# Ensure girder_client installed:
! pip install girder_client

In [1]:
import girder_client
import sys,os,glob

In [2]:
# Subject Data location:
subjRootDir = "/data/HCP_BedpostData/"

# Source subject nodiff:
diffData = "/T1w/Diffusion/nodif*"

# Source ROIs
dtiSpaceMasks = "addlInfo/%s/DTI_ROIs/*"

In [15]:
# Admin login:
apiUrl="http://candygram.neurology.emory.edu:8080/api/v1"
cbGirder = girder_client.GirderClient(apiUrl=apiUrl)
cbGirder.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [4]:
# Destination of Subject data in DTI Space :
subjDTIPath = "/collection/FG/HCP_TractographyData/SubjectRegistrationResults/DTI_Space"
subjDtiTarget = cbGirder.get("resource/lookup?path=" + subjDTIPath)

In [5]:
# List of Subjects:
subjList = [x for x in os.listdir(subjRootDir) if 'addlInfo' not in x]
print(len(subjList),"Subjects have Bedpost Data")

1009 Subjects have Bedpost Data


# Define list of images in subject folder metadata:
dtiFilesForPapaya: Specifies load order of images in subject folder and subfolders

In [74]:
# DTI base data: first: nodif, second: nodif_brain, third: nodif_brain_mask
dtiFilesForPapaya = ['nodif.nii.gz',
                     'nodif_brain.nii.gz',
                     'nodif_brain_mask.nii.gz',
                     'Human_BA17_bilat_trans.nii.gz', 
                     'Human_BA_4ap_bilat_trans.nii.gz', 
                     'Human_BasalForebrain_Bilat_trans.nii.gz', 
                     'Human_BasalForebrain_Left_trans.nii.gz', 
                     'Human_BasalForebrain_Right_trans.nii.gz', 
                     'Human_Bilat_Accumbens_HarvardOxford-sub-maxprob-thr25-1mm_trans.nii.gz', 
                     'Human_Hypothalamus_Bilat_trans.nii.gz', 
                     'Human_Hypothalamus_Left_trans.nii.gz', 
                     'Human_Hypothalamus_Right_trans.nii.gz'
                    ]

In [75]:
# Iterate through subjects:
for s in subjList:
    print('S: %s' % s)
    
    # Reset variables:
    papayaSpecDataDict = {}
    papayaSpec={}
    imageNames = []
    imageList = []
    
    # Subject folder:
    grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))
    # Images in subject folder:
    itemsInSubjFolder = list(cbGirder.listItem(grdrTgtSubjFolder[0]['_id']))
    
    # Subject subfolders: Ex. 'DTI_ROIs'
    foldersInSubjFolder = list(cbGirder.listFolder(grdrTgtSubjFolder[0]['_id']) )
    
    # Iterate through itemsInSubjFolder:
    for i in itemsInSubjFolder:   
        # Image location:                    
        imageURL = apiUrl +'/item/' +i['_id'] + '/download?contentDisposition=attachment'
        #print(imageURL)
        imageList.append( imageURL )
        # Image name:
        imageNames.append(i['name'])
        
    # Iterate through subfolders (Ex. 'DTI_ROIs'):
    for subFolder in foldersInSubjFolder:
        # List of subfolder images:
        subfolderImages = list(cbGirder.listItem(subFolder['_id']) )

        # Iterate through subfolder images:
        for subImage in subfolderImages:
            # Image location:
            imageURL = apiUrl +'/item/' +subImage['_id'] + '/download?contentDisposition=attachment'
            #print(imageURL)
            imageList.append( imageURL )
            # Image name:
            imageNames.append(subImage['name'])
            
    # Order imageList and iamgeNames according to dtiFilesForPapaya:
    indices=[]
    for i in imageNames:
        indices.append( dtiFilesForPapaya.index(i) )
    
    imageNames = [imageNames[x] for x in indices]
    imageList = [imageList[x] for x in indices]
    
    #print(imageNames)
    #print(imageList)
    
    # Upload metadata to Subject folder:        
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"imageList":imageList})
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"imageNames":imageNames})
    

S: 379657
S: 112516
S: 208327
S: 210011
S: 138130
S: 209228
S: 989987
S: 901038
S: 856463
S: 672756
S: 169343
S: 152831
S: 182840
S: 541943
S: 713239
S: 146634
S: 978578
S: 193239
S: 336841
S: 165032
S: 176037
S: 614439
S: 157437
S: 309636
S: 103212
S: 286347
S: 173940
S: 156334
S: 206727
S: 211316
S: 149337
S: 172433
S: 206828
S: 154734
S: 115724
S: 102513
S: 153126
S: 161832
S: 760551
S: 578158
S: 214019
S: 522434
S: 124624
S: 753251
S: 786569
S: 899885
S: 348545
S: 103010
S: 165840
S: 176542
S: 397861
S: 212823
S: 175237
S: 680957
S: 320826
S: 757764
S: 154330
S: 165941
S: 510225
S: 779370
S: 316835
S: 101006
S: 169747
S: 237334
S: 992673
S: 132118
S: 219231
S: 148133
S: 767464
S: 611938
S: 668361
S: 704238
S: 248339
S: 715041
S: 809252
S: 195849
S: 307127
S: 304020
S: 159340
S: 122822
S: 749361
S: 204622
S: 555651
S: 106016
S: 891667
S: 299760
S: 150019
S: 211922
S: 128026
S: 709551
S: 194746
S: 268749
S: 456346
S: 613538
S: 196851
S: 361941
S: 144428
S: 453542
S: 984472
S: 339847


IndexError: list index out of range

# Upload Specification file as metadata for each subject folder:
### Spec defines data visualized parameters and load order

In [76]:
import json

In [93]:
# Import specification file (randomise_output_aesthetic.js):
specFile = '../src/specification_randomise.json'

# Load json into variable:
with open(specFile, 'r') as f:
    specFileData = json.load(f)

In [94]:
# Iterate through subjects:
for s in subjList:
    print('S: %s' % s)
    papayaSpecDataDict = {}
    papayaSpec={}
    
    # Subject folder:
    grdrTgtSubjFolder = list(cbGirder.listFolder(subjDtiTarget['_id'],
                                                 name=s))
    # Upload metadata to Subject folder:        
    cbGirder.addMetadataToFolder(grdrTgtSubjFolder[0]['_id'], {"papayaSpec":specFileData})
    

S: 379657
S: 112516
S: 208327
S: 210011
S: 138130
S: 209228
S: 989987
S: 901038
S: 856463
S: 672756
S: 169343
S: 152831
S: 182840
S: 541943
S: 713239
S: 146634
S: 978578
S: 193239
S: 336841
S: 165032
S: 176037
S: 614439
S: 157437
S: 309636
S: 103212
S: 286347
S: 173940
S: 156334
S: 206727
S: 211316
S: 149337
S: 172433
S: 206828
S: 154734
S: 115724
S: 102513
S: 153126
S: 161832
S: 760551
S: 578158
S: 214019
S: 522434
S: 124624
S: 753251
S: 786569
S: 899885
S: 348545
S: 103010
S: 165840
S: 176542
S: 397861
S: 212823
S: 175237
S: 680957
S: 320826
S: 757764
S: 154330
S: 165941
S: 510225
S: 779370
S: 316835
S: 101006
S: 169747
S: 237334
S: 992673
S: 132118
S: 219231
S: 148133
S: 767464
S: 611938
S: 668361
S: 704238
S: 248339
S: 715041
S: 809252
S: 195849
S: 307127
S: 304020
S: 159340
S: 122822
S: 749361
S: 204622
S: 555651
S: 106016
S: 891667
S: 299760
S: 150019
S: 211922
S: 128026
S: 709551
S: 194746
S: 268749
S: 456346
S: 613538
S: 196851
S: 361941
S: 144428
S: 453542
S: 984472
S: 339847


S: 121416
S: 627852
S: 480141
S: 872562
S: 164939
S: 133928
S: 130417
S: 856968
S: 250932
S: 667056
S: 679770
S: 198653
S: 867468
S: 178950
S: 886674
S: 994273
S: 792867
S: 102311
S: 581349
S: 209935
S: 707749
S: 183337
S: 352738
S: 114116
S: 167743
S: 161731
S: 187850
S: 201111
S: 163129
S: 723141
S: 124422
S: 257845
S: 101915
S: 996782
S: 249947
S: 152427
S: 308129
S: 360030
S: 664757
S: 171532
S: 849971
S: 894067
S: 119025
S: 644246
S: 742549
S: 185442
S: 832651
S: 210617
S: 820745
S: 519647
S: 783462
S: 657659
S: 102008
S: 346137
S: 800941
S: 594156
S: 133625
S: 212015
S: 995174
S: 419239
S: 729557
S: 102614
S: 172130
S: 701535
S: 275645
S: 766563
S: 350330
S: 381038
S: 127731
S: 366042
S: 654754
S: 784565
S: 748258
S: 576255
S: 134829
S: 200109
S: 118023
S: 214221
S: 878877
S: 604537
S: 180836
S: 206929
S: 137431
S: 104820
S: 459453
S: 349244
S: 105216
S: 654552
S: 955465
S: 183034
S: 130316
S: 173637
S: 329844
S: 529953
S: 169444
S: 151324
S: 737960
S: 580347
S: 104416
S: 208428
